In [1]:
from kan.hypothesis import *
from torchvision import transforms
import torch
import torch.nn.functional as F
from kan import *
import h5py
import copy
import sympy as sp
dtype = torch.get_default_dtype()
from sympy import latex
import pickle

### We first check whether perturbation theory works for the lowest band in our original dataset

In [2]:
data_dir = "./"
filename = "sg2-data.h5"
checkpoint_dir = "./saved_models/"
log_dir = './logs/'
band_idx = 0
gidxs = [1, 2, 3, 4]
only_obstructed = False
only_topological = False
binary_classification = False#True
no_penalize_last = True
bias = False
numrs = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
file = h5py.File(data_dir + filename, 'r')
print("file keys: ", file['sg2/1/'].keys());

file keys:  <KeysViewHDF5 ['epsilon_Gs-gidx=1', 'epsilon_Gs-gidx=2', 'epsilon_Gs-gidx=3', 'epsilon_Gs-gidx=4', 'frequencies-gidx=1-mode=tm', 'frequencies-gidx=2-mode=tm', 'frequencies-gidx=3-mode=tm', 'frequencies-gidx=4-mode=tm', 'symmetry-gidx=1-mode=tm', 'symmetry-gidx=2-mode=tm', 'symmetry-gidx=3-mode=tm', 'symmetry-gidx=4-mode=tm']>


In [5]:
nGs = 10
gidxs = [1, 2, 3, 4]
wps = ['1a', '1b', '1c', '1d']
wps2 = ['1a', '1c', '1b', '1d'] # switch 1b and 1c due to convention of how Gvectors are stored

symmetry_data = torch.zeros(10000 * len(gidxs) * len(wps))
input_data = torch.zeros(10000 * len(gidxs) * len(wps), nGs)

In [6]:
for (widx, (wp1, wp2)) in enumerate(zip(wps, wps2)):
    print(f"Wyckoff index: {widx}")
    sym_vec_phases = file[f'sg2/symmetry_vector_phases/{wp1}'][()]
    epsilon_G_phases = file[f'sg2/epsilon_G_phases/{wp2}'][()].real
    for gidx in gidxs:
        print(f"gidx: {gidx}")
        for id in range(1, 10001):
            real_id = (id-1) + (gidx-1)*10000 + len(gidxs)*10000*widx
            if (id % 1000 == 0): 
                print(id)
            symmetry_before_aug = file[f'sg2/{id}/symmetry-gidx={gidx}-mode=tm'][()][band_idx]
            if band_idx == 0:
                symmetry_data[real_id] =  sym_vec_phases[symmetry_before_aug] - 8
            else: 
                symmetry_data[real_id] =  sym_vec_phases[symmetry_before_aug] 

            fourier_data_before_aug = file[f'sg2/{id}/epsilon_Gs-gidx={gidx}'][()][0:nGs].real
            fourier_data = fourier_data_before_aug * epsilon_G_phases
            input_data[real_id, :] = torch.tensor([*fourier_data])

Wyckoff index: 0
gidx: 1
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 2
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 3
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 4
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
Wyckoff index: 1
gidx: 1
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 2
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 3
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 4
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
Wyckoff index: 2
gidx: 1
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 2
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 3
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 4
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
Wyckoff index: 3
gidx: 1
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 2
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 3
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 4
1000
2000
3000
4000
5000
6000
7000
8000

In [8]:
input_data

torch.Size([160000, 10])

### Recall that the embedding is as follows:  8*Γ_irreps + 4*Y_irreps + 2*A_irreps + B_irreps

In [65]:
def perturbation_prediction(x):
    x_important = x[1:4]
    result = 4*(x_important[0]/np.abs(x_important[0])+1)/2 + 2*(x_important[2]/np.abs(x_important[2])+1)/2 + (x_important[1]/np.abs(x_important[1])+1)/2 
    return result

In [66]:
perturbation_results = [perturbation_prediction(input_data[idx, :]).item() for idx in range(0, 160000)]

In [67]:
sum([(x == y).item() for (x, y) in zip(symmetry_data, perturbation_results)])/160000

0.9943

In [51]:
M = 64
xs = ys = np.linspace(-0.5, 0.5, M, endpoint=false)
xs = np.repeat(np.reshape(xs, (1, M)), M, axis=0);
ys = np.repeat(np.reshape(ys, (M, 1)), M, axis=1);
Gs = [[-1, 0], [0, -1], [-1, -1], [-1, 1], [-2, 0], [0, -2], [-2, -1], [-2, 1], [-1, -2]]

def smooth(X, M=64):
    epsilon_grid = np.zeros((M, M))
    for (CG, (Gx, Gy)) in zip(X, Gs):
        epsilon_grid += CG*np.cos(2*np.pi*xs* Gy + 2*np.pi*ys * Gx)
    return epsilon_grid

In [54]:
inverse_design_dir = "./inverse_design_smooth_symbols_random/"
inverse_design_dir_1 = "./inverse_design_smooth_symbols/"

N = 2000
two_tone_classifications = np.zeros((N, 8));

all_data = np.zeros((N, 9))

for topological_class in range(0, 8): 
    i_start = topological_class * 250 
    i_end = i_start + 250 
    filename = f"inverse_design_params-class{topological_class}.h5"
    file = h5py.File(inverse_design_dir + filename, 'r')
    all_data[i_start:i_end, 0:3] = file["inverse_design_params"][()]
    file.close()
    
    filename = f"inverse_design_params-random-class{topological_class}.h5"
    file = h5py.File(inverse_design_dir_1 + filename, 'r')
    all_data[i_start:i_end, 3:9] = file["inverse_design_params_random"][()]
    file.close()

In [84]:
smooth_inverse_design_perturbation_results = [perturbation_prediction([20, *all_data[i, :]]) for i in range(0,2000)];

In [86]:
sum([x == y for (x, y) in  zip(np.hstack([np.repeat(i, 250) for i in range(0, 8)]), smooth_inverse_design_perturbation_results)])

2000

### Now we calculate Fourier components for high contrast 

In [92]:
all_data_high_contrast = np.zeros((N, 9))
for n in range(0, N):
    if (n % 100) == 0:
        print(f"n: {n}", flush=true)
    epsilon_grid = smooth(all_data[n,  :])
    perc25, perc50, perc75 = np.percentile(epsilon_grid, [25, 50, 75])
    epsilon_grid_two_tone = np.where(epsilon_grid < perc50, 1, 12)
    Cgs_two_tone = np.zeros(9)
    for (m, G) in enumerate(Gs): 
        Gx, Gy = G
        G_grid = np.cos(2*np.pi*xs* Gy + 2*np.pi*ys * Gx)
        Cgs_two_tone[m] = ((G_grid * epsilon_grid_two_tone)*2/M/M).sum()
    all_data_high_contrast[n, :] = Cgs_two_tone

n: 0
n: 100
n: 200
n: 300
n: 400
n: 500
n: 600
n: 700
n: 800
n: 900
n: 1000
n: 1100
n: 1200
n: 1300
n: 1400
n: 1500
n: 1600
n: 1700
n: 1800
n: 1900


In [153]:
dir = "./inverse_design_two_tone_symbols_random/";
expect_to_be_good = np.array([], dtype=int)
for n in range(0, 8):
    filename = f"two_tone_scores-class{n}.h5"
    file = h5py.File(dir + filename, 'r')
    #which_good = list(filter(lambda x: x == n, np.argmax(file["two_tone_classifications"][()], axis=1)))

    which_good = np.array([idx + 250 * n for (idx, pred) in enumerate(np.argmax(file["two_tone_classifications"][()], axis=1)) if pred == n])
    print(which_good.shape)
    expect_to_be_good = np.append(expect_to_be_good, which_good);

(200,)
(243,)
(212,)
(212,)
(218,)
(221,)
(235,)
(230,)


In [156]:
smooth_inverse_design_perturbation_results = [perturbation_prediction([20, *all_data[i, :]]) for i in range(0,2000)];

In [167]:
sum(np.array([x == y for (x, y) in  zip(np.hstack([np.repeat(i, 250) for i in range(0, 8)]), smooth_inverse_design_perturbation_results)])[expect_to_be_good])

1755

In [169]:
expect_to_be_good.shape

(1771,)

In [170]:
1755/1771

0.9909655561829475